# 比赛整体的思路：

1、看比赛的背景介绍，比赛的评分方式，主办方对所有字段含义的解释（如果是匿名赛就更复杂了，好在这次算是半匿名，只有部分特征是含义不清的），查看过去相似的比赛以及获奖选手的思路分享与开源代码， 很多时候别人发布过的代码可以直接快速的复用；
2、eda 数据探索，通过可视化、计算统计值等寻找数据中的异常之处，并且在这个过程中进一步熟悉手头的数据，在这个过程中可以参考一些开源的kernel，但是尽量自己思考，而不是直接抄袭，参加比赛的主要目的是学习与自我提升，要建立起自己的思维框架来解决问题；
3、性能优化，包括了原始数据的数据类型的优化以降低内存占用与存储、一些常用的函数的定义避免过于冗余的代码、一些运行时间很久的函数的性能优化以提高试验的速度与效率，文件快速的读取与存储、原始数据的采样等等；
4、建立一个baseline，后续在这个baseline上进行深入的调整，如果实在时间着急，找一个比较恰当合适的baseline；
5、确定一个好的cv策略（对于无序的问题，可能cv就是简单的kfold或者stratifiedkfold，但是对于时序相关的问题，cv的选择还非常较重要的，如何尽量降低时间穿越的影响是一件很重要的事）；
6、根据模型的表现对症下药找到问题所在然后实施对应的特征工程方法衍生新的特征、转换原始特征或者进行特征选择等；
7、在这个过程中不断的阅读新的kernel和discussion来寻找新的灵感
8、使用stacking等集成方法与后处理技巧等方法来上分

# 性能优化，提高建模效率：

大限度减少pandas的内存占用 by 使用更少内存占用的数据类型

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

对于类别特征，还有更好的处理思路：类别就行lable encoder，用int型数据存放类别特征，大大减少内存占用

In [ ]:
# LABEL ENCODE
def encode_LE(col,train=None,test=None,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    nm = col
    if df_comb.max()>32000: 
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
    else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; 
    gc.collect()
    if verbose: print(nm,', ',end='')

或者

In [ ]:
for f in df_train.drop('isFraud', axis=1).columns:
    if df_train[f].dtype=='object' or df_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_train[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_test[f] = lbl.transform(list(df_test[f].values))   

使用numba提速 评价指标的计算，也可用于其它指标

In [ ]:
from numba import jit
@jit
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc

def eval_auc(y_true, y_pred):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true, y_pred), True

# 相关性处理的正确姿势，通过将一些常用的功能编写进入函数实现快速的相关特征的删除

In [ ]:
import gc
import pandas as pd
import numpy as np


def nan_search(data=None):
    nans_df = data.isna()
    nans_groups={}
    for col in data.columns:
        cur_group = nans_df[col].sum()
        try:
            nans_groups[cur_group].append(col)
        except:
            nans_groups[cur_group]=[col]
    del nans_df; 
    gc.collect()
    length=data.shape[0]
    groups=[]
    ratio=[]
    for k,v in nans_groups.items():
        
        print('####### NAN count =',k)
        print('####### NAN ratio =',k/length)
        ratio.append(k/length)
        print(v)
        groups.append(v)
        print('\n')
    return groups,ratio
        
def reduce_group(data,cols):
    drops = []
    for g in cols:
        mx = 0; vx = g[0]
        for gg in g:
            n = data[gg].nunique()
            if n>mx:
                mx = n
                vx = gg
        g.remove(vx)
        drops.append(g)
    print('drop these',drops)
    return drops




def identify_collinear(corr_matrix,correlation_threshold,X):
    
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
    
    # Select the features with correlations above the threshold
    # Need to use the absolute value
    to_drop = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]
    
    # Dataframe to hold correlated pairs
    record_collinear = pd.DataFrame(columns = ['corr_feature1', 'corr_feature2', 'corr_value'])
    
    # Iterate through the columns to drop to record pairs of correlated features
    for column in to_drop:
    
        # Find the correlated features
        corr_features = list(upper.index[upper[column].abs() > correlation_threshold])
    
        # Find the correlated values
        corr_values = list(upper[column][upper[column].abs() > correlation_threshold])
        drop_features = [column for _ in range(len(corr_features))]    
    
        # Record the information (need a temp df for now)
        temp_df = pd.DataFrame.from_dict({'corr_feature1': drop_features,
                                         'corr_feature2': corr_features,
                                         'corr_value': corr_values})
    
        # Add to dataframe
        record_collinear = record_collinear.append(temp_df, ignore_index = True)
    drops=[]## 下面是删除规则
    if record_collinear.shape[0]==0:
        return 'nothing!','nothing'
    for i,j in zip(record_collinear.corr_feature1.values.tolist(),record_collinear.corr_feature2.values.tolist()):
        if X[i].nunique()>X[j].nunique():
            drops.append(j)
        else:
            drops.append(i)
    drops=list(set(drops))
    return record_collinear,drops




# 超参数优化模板
下面列出了最常用的hyperot和bayesaionoptimization，sciki-optimize也很方便，然后就是deap和其对应的sklearn-deap，在实践中，遗传算法没有贝叶斯优化来的好用。


贝叶斯优化之hyperopt

In [ ]:
from sklearn.model_selection import KFold,TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from xgboost import plot_importance
from sklearn.metrics import make_scorer

import time
def objective(params):
    time1 = time.time()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'num_leaves': '{:.3f}'.format(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'min_child_samples': '{:.3f}'.format(params['min_child_samples']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
    }

    print("\n############## New Run ################")
    print(f"params = {params}")
    FOLDS = 7
    count=1
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

    tss = TimeSeriesSplit(n_splits=FOLDS)
    y_preds = np.zeros(sample_submission.shape[0])
    y_oof = np.zeros(X_train.shape[0])
    score_mean = 0
    for tr_idx, val_idx in tss.split(X_train, y_train):
        clf = xgb.XGBClassifier(
            n_estimators=600, random_state=4, verbose=True, 
            tree_method='gpu_hist', 
            **params
        )

        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        
        clf.fit(X_tr, y_tr)
        #y_pred_train = clf.predict_proba(X_vl)[:,1]
        #print(y_pred_train)
        score = make_scorer(roc_auc_score, needs_proba=True)(clf, X_vl, y_vl)
        # plt.show()
        score_mean += score
        print(f'{count} CV - score: {round(score, 4)}')
        count += 1
    time2 = time.time() - time1
    print(f"Total Time Run: {round(time2 / 60,2)}")
    gc.collect()
    print(f'Mean ROC_AUC: {score_mean / FOLDS}')
    del X_tr, X_vl, y_tr, y_vl, clf, score
    return -(score_mean / FOLDS)


space = {
    # The maximum depth of a tree, same as GBM.
    # Used to control over-fitting as higher depth will allow model 
    # to learn relations very specific to a particular sample.
    # Should be tuned using CV.
    # Typical values: 3-10
    'max_depth': hp.quniform('max_depth', 7, 23, 1),
    
    # reg_alpha: L1 regularization term. L1 regularization encourages sparsity 
    # (meaning pulling weights to 0). It can be more useful when the objective
    # is logistic regression since you might need help with feature selection.
    'reg_alpha':  hp.uniform('reg_alpha', 0.01, 0.4),
    
    # reg_lambda: L2 regularization term. L2 encourages smaller weights, this
    # approach can be more useful in tree-models where zeroing 
    # features might not make much sense.
    'reg_lambda': hp.uniform('reg_lambda', 0.01, .4),
    
    # eta: Analogous to learning rate in GBM
    # Makes the model more robust by shrinking the weights on each step
    # Typical final values to be used: 0.01-0.2
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    
    # colsample_bytree: Similar to max_features in GBM. Denotes the 
    # fraction of columns to be randomly samples for each tree.
    # Typical values: 0.5-1
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, .9),
    
    # A node is split only when the resulting split gives a positive
    # reduction in the loss function. Gamma specifies the 
    # minimum loss reduction required to make a split.
    # Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
    'gamma': hp.uniform('gamma', 0.01, .7),
    
    # more increases accuracy, but may lead to overfitting.
    # num_leaves: the number of leaf nodes to use. Having a large number 
    # of leaves will improve accuracy, but will also lead to overfitting.
    'num_leaves': hp.choice('num_leaves', list(range(20, 250, 10))),
    
    # specifies the minimum samples per leaf node.
    # the minimum number of samples (data) to group into a leaf. 
    # The parameter can greatly assist with overfitting: larger sample
    # sizes per leaf will reduce overfitting (but may lead to under-fitting).
    'min_child_samples': hp.choice('min_child_samples', list(range(100, 250, 10))),
    
    # subsample: represents a fraction of the rows (observations) to be 
    # considered when building each subtree. Tianqi Chen and Carlos Guestrin
    # in their paper A Scalable Tree Boosting System recommend 
    'subsample': hp.choice('subsample', [0.2, 0.4, 0.5, 0.6, 0.7, .8, .9]),
    
    # randomly select a fraction of the features.
    # feature_fraction: controls the subsampling of features used
    # for training (as opposed to subsampling the actual training data in 
    # the case of bagging). Smaller fractions reduce overfitting.
    'feature_fraction': hp.uniform('feature_fraction', 0.4, .8),
    
    # randomly bag or subsample training data.
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, .9)
    
    # bagging_fraction and bagging_freq: enables bagging (subsampling) 
    # of the training data. Both values need to be set for bagging to be used.
    # The frequency controls how often (iteration) bagging is used. Smaller
    # fractions and frequencies reduce overfitting.
}

# Set algoritm parameters
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=27)

# Print best parameters
best_params = space_eval(space, best)

贝叶斯优化之基于高斯过程的 贝叶斯优化

In [ ]:
def LGB_bayesian(
    #learning_rate,
    num_leaves, 
    bagging_fraction,
    feature_fraction,
    min_child_weight, 
    min_data_in_leaf,
    max_depth,
    reg_alpha,
    reg_lambda
     ):
    
    # LightGBM expects next three parameters need to be integer. 
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int
    

    param = {
              'num_leaves': num_leaves, 
              'min_data_in_leaf': min_data_in_leaf,
              'min_child_weight': min_child_weight,
              'bagging_fraction' : bagging_fraction,
              'feature_fraction' : feature_fraction,
              #'learning_rate' : learning_rate,
              'max_depth': max_depth,
              'reg_alpha': reg_alpha,
              'reg_lambda': reg_lambda,
              'objective': 'binary',
              'save_binary': True,
              'seed': 1337,
              'feature_fraction_seed': 1337,
              'bagging_seed': 1337,
              'drop_seed': 1337,
              'data_random_seed': 1337,
              'boosting_type': 'gbdt',
              'verbose': 1,
              'is_unbalance': False,
              'boost_from_average': True,
              'metric':'auc'}    
    
    oof = np.zeros(len(train_df))
    trn_data= lgb.Dataset(train_df.iloc[bayesian_tr_idx][features].values, label=train_df.iloc[bayesian_tr_idx][target].values)
    val_data= lgb.Dataset(train_df.iloc[bayesian_val_idx][features].values, label=train_df.iloc[bayesian_val_idx][target].values)

    clf = lgb.train(param, trn_data,  num_boost_round=50, valid_sets = [trn_data, val_data], verbose_eval=0, early_stopping_rounds = 50)
    
    oof[bayesian_val_idx]  = clf.predict(train_df.iloc[bayesian_val_idx][features].values, num_iteration=clf.best_iteration)  
    
    score = roc_auc_score(train_df.iloc[bayesian_val_idx][target].values, oof[bayesian_val_idx])

    return score


bounds_LGB = {
    'num_leaves': (31, 500), 
    'min_data_in_leaf': (20, 200),
    'bagging_fraction' : (0.1, 0.9),
    'feature_fraction' : (0.1, 0.9),
    #'learning_rate': (0.01, 0.3),
    'min_child_weight': (0.00001, 0.01),   
    'reg_alpha': (1, 2), 
    'reg_lambda': (1, 2),
    'max_depth':(-1,50),
}


LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=42)


In [ ]:
init_points = 10
n_iter = 15
print('-' * 130)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)
    
    
LGB_BO.max['target']
LGB_BO.max['params']

遗传算法 deap

In [ ]:
import random
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score

from deap import base
from deap import creator
from deap import tools

#######################
# Load & process data
#######################
def get_data():
    #################
    # read datasets
    #################
    train = pd.read_csv('../input/train.csv')
    test_submit = pd.read_csv('../input/test.csv')

    # Get y and ID
    train = train[train.y < 250] # Optional: Drop y outliers
    y_train = train['y']
    train = train.drop('y', 1)
    test_submit_id = test_submit['ID']

    #########################
    # Create data
    #########################
    features = ['X0',
                'X5',
                'X118',
                'X127',
                'X47',
                'X315',
                'X311',
                'X179',
                'X314',
                'X232',
                'X29',
                'X263',
                'X261']

    # Build a new dataset using key parameters, lots of drops
    train = train[features]
    test_submit = test_submit[features]

    # Label encoder
    for c in train.columns:
        if train[c].dtype == 'object':
            lbl = LabelEncoder()
            lbl.fit(list(train[c].values) + list(test_submit[c].values))
            train[c] = lbl.transform(list(train[c].values))
            test_submit[c] = lbl.transform(list(test_submit[c].values))

    # Convert to matrix
    train = train.as_matrix()
    y_train = np.transpose([y_train.as_matrix()])
    test_submit = test_submit.as_matrix()
    test_submit_id = test_submit_id.as_matrix()

    return train, y_train, test_submit, test_submit_id

#########################
# XGBoost Model
#########################
def gradient_boost(train, y_train, params):
    y_mean_train = np.mean(y_train)

    # prepare dict of params for xgboost to run with
    xgb_params = {
        'n_trees': params[0],
        'eta': params[1],
        'max_depth': params[2],
        'subsample': params[3],
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'base_score': y_mean_train,
        'seed': 123456789,
        'silent': 1
    }

    # form DMatrices for Xgboost training
    dtrain = xgb.DMatrix(train, y_train)

    # xgboost, cross-validation
    cv_result = xgb.cv(xgb_params,
                       dtrain,
                       nfold = 10,
                       num_boost_round=5000,
                       early_stopping_rounds=100,
                       verbose_eval=False,
                       show_stdv=False
                      )

    num_boost_rounds = len(cv_result)

    # train model
    model = xgb.train(dict(xgb_params), dtrain, num_boost_round=num_boost_rounds)

    # get model accuracy
    accuracy = r2_score(dtrain.get_label(), model.predict(dtrain))

    return model, accuracy

######################
# Genetic algorithm
######################
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Get data
train, y_train, test_submit, test_submit_id = get_data()

# Attribute generator
toolbox.register("n_trees", random.randint, 100, 10000)
toolbox.register("eta", random.uniform, 0.0001, 0.01)
toolbox.register("max_depth", random.randint, 1, 10)
toolbox.register("subsample", random.uniform, 0, 1)

# Structure initializers
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.n_trees, toolbox.eta, toolbox.max_depth, toolbox.subsample), n=1)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ('fitness') function to be maximized
def evalOneMax(individual):
    model, accuracy = gradient_boost(train, y_train, individual)
    return [accuracy]

# the model for a specified individual
def getModel(individual):
    model, accuracy = gradient_boost(train, y_train, individual)
    return model

# register the goal / fitness function
toolbox.register("evaluate", evalOneMax)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation
toolbox.register("select", tools.selTournament, tournsize=3)


def main():
    random.seed(12345)

    # create an initial population
    pop = toolbox.population(n=20)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    while max(fits) < 100 and g < 1000:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x * x for x in fits)
        std = abs(sum2 / length - mean ** 2) ** 0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

        best_ind = tools.selBest(pop, 1)[0]
        print("Best individual so far is %s, %s" % (best_ind, best_ind.fitness.values))

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

if __name__ == "__main__":
    main()

# 快速固定随机性

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

# 最常见的特征衍生通用方案，结合feature importance来衍生

In [ ]:
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col+'_FE'
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype('float32')
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype('float32')
        print(nm,', ',end='')
        
# LABEL ENCODE
def encode_LE(col,train=X_train,test=X_test,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    nm = col
    if df_comb.max()>32000: 
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
    else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()
    if verbose: print(nm,', ',end='')
        
# GROUP AGGREGATION MEAN AND STD
# https://www.kaggle.com/kyakovlev/ieee-fe-with-some-eda
def encode_AG(main_columns, uids, aggregations=['mean'], train_df=X_train, test_df=X_test, 
              fillna=True, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')
                test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')
                
                if fillna:
                    train_df[new_col_name].fillna(-1,inplace=True)
                    test_df[new_col_name].fillna(-1,inplace=True)
                
                print("'"+new_col_name+"'",', ',end='')
                
# COMBINE FEATURES
def encode_CB(col1,col2,df1=X_train,df2=X_test):
    nm = col1+'_'+col2
    df1[nm] = df1[col1].astype(str)+'_'+df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    encode_LE(nm,verbose=False)
    print(nm,', ',end='')
    
# GROUP AGGREGATION NUNIQUE
def encode_AG2(main_columns, uids, train_df=X_train, test_df=X_test):
    for main_column in main_columns:  
        for col in uids:
            comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
            print(col+'_'+main_column+'_ct, ',end='')

# 目前kaggle上predict 清一色的套路方法，不再是单模而是交叉验证形成多个模型然后集成输出

In [ ]:
for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
    start_time = time()
    print('Training on fold {}'.format(fold + 1))
    
    trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
    clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
    
    feature_importances['fold_{}'.format(fold + 1)] = clf.feature_importance()
    aucs.append(clf.best_score['valid_1']['auc'])
    
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))

# 两个经典工具模板，给了一个大框架写了一些具体的实例，可以根据自己的需求自己设计一些用的顺手的工具或者把流程性的代码保存下来等

https://github.com/Ynakatsuka/kaggle_utils   工具模板

https://github.com/Yimeng-Zhang/feature-engineering-and-feature-selection   流程性代码

#  特征偏移的检验与缓解办法

（1）、检验方法有对抗性验证、kris的验证法，我之前尝试过用psi这类过滤式的指标来衡量特征的偏移程度，但是整体上没有kris的验证方法来的直观，因为这种验证方法直接将特征的偏移程度通过我们关心的评价指标反应出来

（2）、一些常用的缓解方法
连续特征：
1、对数变换等，对数变换属于box-cox变换的一种，sklearn中的Transform功能或者scipy的box-cox功能可实现这类变换；
2、分箱，有效改善特征偏移问题，但是需要作为超参数调整比如分箱数量，分箱算法选择等，比较麻烦，可以考虑结合kris验证方法，生成新的分箱特征之后测试一下泛化误差的大小；
3、加减乘除，例如时间相关特征可以考虑与时间特征做差或者做除来消除时间相关性；
4、待补充

类别特征：
1、少数类别合并；
2、类别转int类型特征；
3、类别特征编码并删除原特征或者原特征转连续特征；
4、待补充